##Import the library

*pandas
*numpy
*matplotlib
*seaborn
*sklearn

In [1]:
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import sklearnex

from xgboost import XGBClassifier
from xgboost import XGBRegressor

# INTEL sklearn patching for optimizing sklearn
from sklearnex import patch_sklearn
patch_sklearn(global_patch=True)

# Reimport sklearn
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import mutual_info_regression

# Modin, faster alternative compare to Pandas

import modin.pandas as pd
import modin.config as cfg

cfg.Engine.put('Dask')
cfg.Memory.put(9000000000)
#cfg.CpuCount.put(4)

#Dask
from distributed import Client
client = Client()

from pprint import pprint

#Ray
# import ray
# ray.init()

Scikit-learn was successfully globally patched by Intel(R) Extension for Scikit-learn


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
2022-09-17 16:23:54,689 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-25h36xaq', purging
2022-09-17 16:23:54,689 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-onb7rfsi', purging
2022-09-17 16:23:54,689 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-z895v1aw', purging


#Import the data
*train data
*test data

In [2]:
# # check the number CPU
# import modin
# print(modin.config.NPartitions.get())

8


In [3]:
# Check Dask client status
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:45393/status,
Dashboard: http://127.0.0.1:45393/status,Workers: 4
Total threads: 8,Total memory: 15.48 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36219,Workers: 4
Dashboard: http://127.0.0.1:45393/status,Total threads: 8
Started: Just now,Total memory: 15.48 GiB
Comm: tcp://127.0.0.1:45193,Total threads: 2
Dashboard: http://127.0.0.1:45103/status,Memory: 3.87 GiB
Nanny: tcp://127.0.0.1:45275,


In [4]:
train_data = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/train_data.csv'


In [5]:
test_data = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/test_data.csv'

In [6]:
train_hdf5 = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/train_data.csv.hdf5'

In [7]:
to_parquet_file = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_train'

In [8]:
to_parquet_file_test = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/df_parquet_test'

In [9]:
train_labels = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/amex-default-prediction/train_labels.csv'

In [10]:
train_labels_file = '/mnt/b7b917e1-da96-4995-b7db-c30035d41dbe/Machine Learning Project/AMEX_2022/train_labels_file'

In [11]:
# pd.read_hdf(train_hdf5)

In [12]:
# df_empty = pd.DataFrame()

In [13]:
import dask.dataframe as dd

## Convert csv to parquet for more efficient read/write

In [14]:
# df = dd.read_csv(train_data)
df = dd.read_csv(train_labels)

In [15]:
df = df.repartition(partition_size='200MB')
df.to_parquet(train_labels_file, write_index=False)

[None]

In [16]:
df = dd.read_parquet(train_labels_file)

In [17]:
df = pd.read_parquet(train_labels_file)

In [18]:
df.head()

,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0
